In [39]:
pip install ortools


In [40]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import random
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import time

In [42]:
def fit(inv, dis_mat, n):
    summ = np.sum([dis_mat[int(inv[i])][int(inv[i+1])] for i in range(2*n-1)])
    return summ + dis_mat[0][int(inv[0])] + dis_mat[int(inv[-1])][0]

def input_from_text(path):
    f = open(path, "r")
    lines = f.readlines()
    n, k = int(lines[0].split()[0]), int(lines[0].split()[1])
    c = []
    for line in lines[1:-2]:
        c.append([float(c_i) for c_i in line.split()])
    f.close()
    return n, k, c, fit([int(c_i) for c_i in lines[-1].split()], dis_mat=c, n=n)

def Input():
    line1 = input().split()
    n, k = [int(i) for i in line1]
    c = []
    for i in range(2*n + 1):
        str_ci = input().split()
        c_i = [int(j) for j in str_ci]
        c.append(c_i)
    return n, k, c

def nOfBlindingConstrains(inv):
    count = 0
    mark = [0]*(n+1)
    n_passengers = 0
    for i in range(2*n):
        if inv[i] <= n:
            n_passengers += 1
            mark[inv[i]] = 1
        else:
            n_passengers -= 1
            mark[inv[i]-n] = 0
        if n_passengers > k:
            count += 1

    return count + np.sum(mark)

def generate_dataset(n, max_w, seed):
    random.seed(seed)
    dis_mat = [[0 for i in range(0, 2*n+1)] for j in range(0, 2*n+1)]
    for i in range(0, 2*n+1):
        for j in range(i+1, 2*n+1):
            dis_mat[i][j] = randint(1, max_w)
            dis_mat[j][i] = dis_mat[i][j]
    return dis_mat

In [43]:
n, k, c, erp_ov = input_from_text("/content/drive/MyDrive/TULKH/n5_k3.txt")
erp_ov

49.0

In [44]:
# gerate random dataset
n, k = 5, 3
c = generate_dataset(n, 50, 1)

# **Branch and Bound**

In [45]:
x = np.zeros(2*n + 1, dtype=int)        # [0, 0, 0,...0, 0]
visited = np.zeros(2*n + 1, dtype=bool) # [False, False,...False, False]
cities = list(np.arange(1, 2*n + 1))    # [1, 2, 3,...2n-1, 2n]
load = 0
best_fitness = 1e9
best_sol = None
d_min = c[0][1]

for i in range(2*n + 1):
    for j in range(2*n + 1):
        if ((c[i][j] < d_min) and (c[i][j] > 0)):
            d_min = c[i][j]

def fitness(j):
    res = c[x[j]][0]
    for idx in range(j):
        res += c[x[idx]][x[idx+1]]

    return res

def check(v):
    if visited[v] == True:
        return False

    # Trả khách
    if v > n:
        if visited[v-n] == True:
            return True
        return False
    # Đón khách
    else:
        if load < k:
            return True
        return False

def Try(j):
    global load
    global best_fitness
    global best_sol

    for v in cities:
        if check(v):
            x[j] = v

            # Cập nhật các ràng buộc cục bộ
            visited[v] = True
            if v <= n:  load += 1
            else:       load -= 1

            # Cập nhật kỷ lục
            if j == 2*n:
                c_fitness = fitness(j)
                if c_fitness < best_fitness:
                    best_fitness = c_fitness
                    best_sol = x.copy()
            else:
                if (fitness(j) - c[x[j]][0] + (2*n - j + 1) * d_min) < best_fitness:
                    Try(j + 1)

            # Khôi phục ràng buộc cục bộ
            visited[v] = False
            if v <= n:  load -= 1
            else:       load += 1
    #end
t1 = time.time()
Try(1)
print(f"Time: {time.time()-t1}")
print(f"Objective value: {best_fitness}")
print(f"Solution: {best_sol}")
print(f"Number of blinding constrains: {nOfBlindingConstrains(best_sol[1:])}")

Time: 0.15632891654968262
Objective value: 108
Solution: [ 0  4  9  3  8  2  5  7  1 10  6]
Number of blinding constrains: 0


# **Constrains Programming**

In [46]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

x = []
for i in range(2*n + 1):
    x_i = []
    for j in range(2*n + 1):
        if i == j:
            x_i.append(0)
        else:
            x_i.append(model.NewIntVar(0, 1, f'x[{i}][{j}]'))
    x.append(x_i)

y = [model.NewIntVar(1, 2*n + 1, f'y[{i}]') for i in range(2*n + 1)]

p = [model.NewIntVar(0, k, f'p[{i}]') for i in range(2*n + 1)]

###### constrains ----------------------------------------------------------
# one way in, one way out
for i in range(2*n + 1):
    model.Add(sum(x[i][j] for j in range(2*n + 1)) == 1)
    model.Add(sum(x[j][i] for j in range(2*n + 1)) == 1)

# start at city 0
model.Add(y[0] == 1)

# n of passenger at city 0 equals 0
model.Add(p[0] == 0)

# not subcycles
for i in range(2*n + 1):
    for j in range(1, 2*n + 1):
        if i == j:
            continue
        model.Add(y[i]+1 == y[j]).OnlyEnforceIf(x[i][j])

# picks before drops
for i in range(n+1, 2*n + 1):
    model.Add(y[i] > y[i - n])

# maximum k passengers
for i in range(2*n + 1):
    for j in range(1, 2*n+1):
        if i == j:
            continue
        # picks
        if j <= n:
            model.Add(p[j]-p[i]==1).OnlyEnforceIf(x[i][j])
        # drops
        if j > n:
            model.Add(p[i]-p[j]==1).OnlyEnforceIf(x[i][j])

###### Objective -------------------------------------------------------
model.Minimize(sum(sum(x[i][j]*c[i][j] for j in range(2*n + 1)) for i in range (2*n + 1)))

###### Solve -----------------------------------------------------------
solver = cp_model.CpSolver()
t1 = time.time()
stt = solver.Solve(model)
print(f"Time: {time.time()-t1}")
if stt == cp_model.OPTIMAL:
    print(f"Objective value: {solver.ObjectiveValue()}")
    sol = {}
    for i in range(2*n + 1):
        sol[solver.Value(y[i])] = i
    sol = [sol[i+1] for i in range(2*n + 1)]
    print(f"Solution: {sol}")
    print(f"Number of blinding constrains: {nOfBlindingConstrains(sol[1:])}")

Time: 0.11661338806152344
Objective value: 108.0
Solution: [0, 4, 9, 3, 8, 2, 5, 7, 1, 10, 6]
Number of blinding constrains: 0


# **Mixed Interger Programing**

In [47]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SCIP")

###### variables -----------------------------------------------------------
x = []
for i in range(2*n + 1):
    x_i = []
    for j in range(2*n + 1):
        if i == j:
            x_i.append(0)
        else:
            x_i.append(solver.IntVar(0, 1, f'x[{i}][{j}]'))
    x.append(x_i)

y = [solver.IntVar(1, 2*n + 1, f'y[{i}]') for i in range(2*n + 1)]

p = [solver.IntVar(0, k, f'p[{i}]') for i in range(2*n + 1)]

###### constrains ----------------------------------------------------------
# one way in, one way out
for i in range(2*n + 1):
    solver.Add(sum(x[i][j] for j in range(2*n + 1)) == 1)
    solver.Add(sum(x[j][i] for j in range(2*n + 1)) == 1)

# start at city 0
solver.Add(y[0] == 1)

# n of passenger at city 0 equals 0
solver.Add(p[0] == 0)

# not subcycles
for i in range(2*n + 1):
    for j in range(1, 2*n + 1):
        if i == j:
            continue
        solver.Add(y[i] - y[j] + (2*n + 1)*x[i][j] <= 2*n)

# picks before drops
for i in range(n+1, 2*n + 1):
    solver.Add(y[i] >= (y[i - n] + 1))

# maximum k passengers
for i in range(2*n + 1):
    for j in range(1, 2*n+1):
        if i == j:
            continue
        # picks
        if j <= n:
            solver.Add(p[j] - p[i] >= 1 - (k + 1)*(1 - x[i][j]))
            solver.Add(p[j] - p[i] <= 1 + (k - 1)*(1 - x[i][j]))
        # drops
        if j > n:
            solver.Add(p[i] - p[j] >= 1 - (k + 1)*(1 - x[i][j]))
            solver.Add(p[i] - p[j] <= 1 + (k - 1)*(1 - x[i][j]))

###### Objective -------------------------------------------------------
solver.Minimize(sum(sum(x[i][j]*c[i][j] for j in range(2*n + 1)) for i in range (2*n + 1)))

###### Solve -----------------------------------------------------------
t1 = time.time()
stt = solver.Solve()
print(f"Time: {time.time()-t1}")
if stt == pywraplp.Solver.OPTIMAL:
    print(f"Objective value: {int(solver.Objective().Value())}")
    sol = {}
    for i in range(2*n + 1):
        sol[y[i].solution_value()] = i
    sol = [sol[i+1] for i in range(2*n + 1)]
    print(f"Solution: {sol}")
    print(f"Number of blinding constrains: {nOfBlindingConstrains(sol[1:])}")

Time: 1.915745735168457
Objective value: 107
Solution: [0, 4, 9, 3, 8, 2, 5, 7, 1, 10, 6]
Number of blinding constrains: 0


# **Greedy**


In [36]:
solution = [] # visit sequence
solution.append(0)

distance = 0 # total travel distance (cost)

visited = []  # array to mark visited point
visited.append(1)
for i in range(2*n):
    visited.append(0)

# function to check if node is valid. Valid if with current solution, number of guest in bus is less than k and target node is not visited. Return True if valid, False if not valid
def is_valid(sol, node: int) -> bool:
    #check if node is visited
    if visited[node] == 1:
        return False

    #construc next solution
    next_sol = sol.copy()
    next_sol.append(node)
    #validate it
    mark = [0]*(n+1)
    n_passengers = 0
    for city in next_sol[1:]:
        if city <= n:
            n_passengers += 1
            mark[city] = 1
        else:
            n_passengers -= 1
            mark[city-n] = 0
        if n_passengers > k:
            # print(sol)
            return False
    if np.sum(mark) - n_passengers != 0:
        # print(sol)
        return False
    return True

def solver(sol):
    for _ in range(1, 2*n + 1):

        #get last_node
        last_node = sol[-1]

        #loop through all possible move from last_node
        possible_move = [] #(distance, node)

        for i in range(1,2*n+1):
            #call function to check if node is valid
            if is_valid(sol, i) == False:
                continue

            #calculate distance and push to possible_move list
            possible_move.append((c[last_node][i], i)) #(distance, node)

        #sort possible_move list by distance, ascending
        possible_move.sort(key=lambda x: x[0])

        #pick first element of possible_move list (shortest distance)
        next_node = possible_move[0][1]

        #update solution
        sol.append(next_node)

        #mark visited
        visited[next_node] = 1

        #update distance
        global distance
        distance += possible_move[0][0]

    #recursion
    return sol

t1 = time.time()
#start recursion
abc = solver(solution)
print(f"Time: {time.time()-t1}")
print("Objective value: ", distance + c[abc[-1]][0])
print("Solution: ", abc)
print(f"Number of blinding constrains: {nOfBlindingConstrains(abc[1:])}")

Time: 0.0009891986846923828
Objective value:  154
Solution:  [0, 4, 1, 5, 9, 3, 8, 2, 10, 6, 7]
Number of blinding constrains: 0


# **GREEDY +**

In [37]:
sqrt_n = np.sqrt(n)

def weight_j(r, i):
    ln = -np.log(r+1e-5)
    if ln < 1e-4:
        return 0
    return np.power(ln, 1 + 50*np.sqrt(i/n))

def valid(sol, city, visited):
    #check if node is visited
    if visited[city] == 1:
        return False

    next_sol = sol.copy()
    next_sol.append(city)

    mark = [0]*(n+1)
    n_passengers = 0
    for _city in next_sol[1:]:
        if _city <= n:
            n_passengers += 1
            mark[_city] = 1
        else:
            n_passengers -= 1
            mark[_city-n] = 0
        if n_passengers > k:
            return False

    if np.sum(mark)-n_passengers != 0:
        return False
    return True

def solve():
    solution = []
    solution.append(0) # start at city 0
    fitness = 0 # objective value
    visited = [0]*(2*n+1)
    visited[0] = 1
    for _ in range(1, 2*n + 1):

        #current city
        c_city = solution[-1]

        weights = [] # probability distribution
        possible_move = []
        d_max = 0

        for city in range(1, 2*n+1):
            if valid(solution, city, visited) == False:
                continue
            d_i = c[c_city][city]
            possible_move.append([city, d_i]) #[city, distance]
            # if d_i > d_max:
            #     d_max = d_i

        possible_move = sorted(possible_move, key=lambda x: x[1])

        if len(possible_move) > 20:
            possible_move = possible_move[:20]

        for city in possible_move:
            if d_max < city[1]: d_max=city[1]

        # print(solution)
        # print(possible_move)
        # print(weights)

        if d_max == 0:
            next_city = possible_move[0]
        else:
            # cal weights
            for move in possible_move:
                r = move[1]/d_max
                weights.append(weight_j(r, _))

            if np.sum(weights) == 0:
                next_city = possible_move[0]
            else:
                # choices next city
                next_city = random.choices(possible_move, k=1, weights=weights)[0]

        #update solution
        solution.append(next_city[0])

        #mark visited
        visited[next_city[0]] = 1

        fitness += next_city[1]

    return solution, fitness

def solver():
    record = 10e9
    best_sol = None
    for seed in range(10):
        print("seed: ", seed)
        random.seed(seed)
        sol, fit = solve()
        print(fit + c[sol[-1]][0])
        if record > fit:
            record = fit
            best_sol = sol.copy()
    return best_sol, record

t1 = time.time()
best_sol, best_fitness = solver()
print(f"Time: {time.time()-t1}")

print("Objective value: ", best_fitness + c[best_sol[-1]][0])
print("Solution: ", best_sol)
print(f"Number of blinding constrains: {nOfBlindingConstrains(best_sol[1:])}")

seed:  0
124
seed:  1
124
seed:  2
124
seed:  3
124
seed:  4
154
seed:  5
124
seed:  6
124
seed:  7
154
seed:  8
124
seed:  9
154
Time: 0.020690441131591797
Objective value:  124
Solution:  [0, 4, 9, 3, 8, 2, 5, 7, 1, 6, 10]
Number of blinding constrains: 0


# **GA with Tabu Search**

In [38]:
dis_mat = c
# hàm tính tổng khoảng cách
def totalDistance(inv):
    summ = np.sum([dis_mat[int(inv[i])][int(inv[i+1])] for i in range(2*n-1)])
    return summ + dis_mat[0][int(inv[0])] + dis_mat[int(inv[-1])][0]

def nOfBlindingConstrains(inv):
    count = 0
    mark = [0]*(n+1)
    n_passengers = 0
    for i in range(2*n):
        if inv[i] <= n:
            n_passengers += 1
            mark[inv[i]] = 1
        else:
            n_passengers -= 1
            mark[inv[i]-n] = 0
        if n_passengers > k:
            count += 1

    return count + np.sum(mark)

def fitness(inv): # O(N)
    pen = totalDistance(np.arange(1, len(inv) + 1))
    distance = totalDistance(inv)
    return 5*pen*nOfBlindingConstrains(inv)/np.sqrt(distance) + distance

# khởi tạo quần thể ban đầu
def initPopulation(cities, size):
    population = []

    for i in range(size):
        invi = list(cities)
        random.shuffle(invi)
        distance = fitness(invi)
        population.append([distance, invi, 0])

    return population

# Toán tử lai 1 điểm cắt
#   father1: 2,3,5|1,4    father2: 1,3,5|4,2     điểm cắt: 3
# new_invi1: 1,3,5,2,4  new_invi2: 2,3,5,1,4
def singlePointCrossover(father1, father2):
    point = random.randint(0, len(father1) - 1)

    new_invi1 = father2[0:point]
    for invi in father1:
        if invi not in new_invi1:
            new_invi1.append(invi)

    new_invi2 = father1[0:point]
    for invi in father2:
        if invi not in new_invi2:
            new_invi2.append(invi)

    return new_invi1, new_invi2

# Toán tử lai 2 điểm cắt
#   father1: 6,2|4,3|1,5    father2: 6,1|3,5|4,2     điểm cắt: 2,4
# new_invi1: 6,2,3,5,4,1  new_invi2: 6,1,4,3,5,2
def multiPointCrossover(father1, father2): # O(N^2)
    points = sorted(random.sample(father1, 2))
    point1 = int(points[0]) - 1
    point2 = int(points[1]) - 1

    new_invi1 = father2[point1:point2]
    c1 = 0
    for invi in father1:
        if invi not in new_invi1:
            c1 += 1
            if c1 <= point1:
                new_invi1.insert(c1-1, invi)
            else:
                new_invi1.append(invi)


    new_invi2 = father1[point1:point2]
    c1 = 0
    for invi in father2:
        if invi not in new_invi2:
            c1 += 1
            if c1 <= point1:
                new_invi2.insert(c1-1, invi)
            else:
                new_invi2.append(invi)

    return new_invi1, new_invi2

def multiPointCrossover1(father1, father2):
    points = sorted(random.sample(father1, 2))
    point1 = int(points[0]) - 1
    point2 = int(points[1]) - 1

    new_invi1 = father2[point1:point2]
    c1 = 0
    c2 = 0
    for invi in father1:
        if invi not in new_invi1:
            c1 += 1
            if c1 <= point1:
                new_invi1.append(invi)
            else:
                new_invi1.insert(c2, invi)
                c2 += 1


    new_invi2 = father1[point1:point2]
    c1 = 0
    c2 = 0
    for invi in father2:
        if invi not in new_invi2:
            c1 += 1
            if c1 <= point1:
                new_invi2.append(invi)
            else:
                new_invi2.insert(c2, invi)
                c2 += 1

    return new_invi1, new_invi2

def nDiffCities(inv1, inv2): # O(N)
    return np.sum(np.array(inv1)!=np.array(inv2))

def chooseFather(population): # O(p*N)
    father1 = random.choice(population)[1]
    tabu = []
    max_diff = 0
    for invi in population:
        diff = nDiffCities(father1, invi[1])
        tabu.append([diff, invi[1]])
        if diff > max_diff:
            max_diff = diff
    threshold = max_diff/2
    nonTabuList = []
    for tabu_i in tabu:
        if tabu_i[0] < threshold:
            continue
        nonTabuList.append(tabu_i[1])
    if len(nonTabuList) == 0:
        return None, None
    return father1, random.choice(nonTabuList)

# đột biến: chọn ngẫu nhiên 2 thành phố rồi đổi chỗ cho nhau, thực hiện 2 lần
# 2,1,5,3,4 => 4,1,5,3,2 => 1,4,5,3,2
def mutation(father, lenCities=51): # O(N)
    new_invi = father.copy()
    for _ in range(5):
        points = random.sample(new_invi, 2)
        temp = new_invi[int(points[0]) - 1]
        new_invi[int(points[0]) - 1] = new_invi[int(points[1]) - 1]
        new_invi[int(points[1]) - 1] = temp

    return new_invi

def tournaments(old_population, new_invivduals, pop_size = 100): # O(plogp)
    merge = sorted(old_population + new_invivduals, key=lambda x: (x[0], -x[2]))
    best_fitness = merge[0][0]
    new_population = merge[:10]
    selected = []
    for inv in merge[10:]:
        if inv[0]/best_fitness > 0.75 or inv[2] < 50:
            selected.append(inv)

    new_population = new_population + random.choices(selected, k=pop_size-10)

    for inv in new_population:
        inv[2] += 1

    return new_population

def GA(
    population,
    lenCities = 51,
    pop_size = 100,
    num_of_generations = 2000,
    crossover_rate = 0.99,
    mutation_rate = 0.2,
    crossover_func=singlePointCrossover
):

    for _ in range(num_of_generations):
        new_inviduals = []
        # sinh pop_size cá thể mới
        n_i = int(pop_size/2)
        for _i in range(n_i):                               # O(g*[p/2*(p*N + N^2 + N) + plogp]) = O(g*p(p*N + N^2 + p*logp))

            father1, father2 = chooseFather(population)
            if father1 != None:
                # xảy ra quá trình lai ghép (crossover)
                if random.random() <= crossover_rate:
                    # thực hiện lai ghép 2 cá thể này
                    inv1, inv2 = crossover_func(father1, father2)
                    # xảy ra quá trình đột biến (mutation)
                    if random.random() <= mutation_rate:
                        inv1 = mutation(inv1, lenCities)
                        inv2 = mutation(inv2, lenCities)
                # không có lai ghép xảy ra
                else:
                    inv1, inv2 = father1, father2
            else:
                return sorted(population)
            new_inviduals.append([fitness(inv1), inv1, 0])
            new_inviduals.append([fitness(inv2), inv2, 0])

        population = tournaments(population, new_inviduals, pop_size)
        population = sorted(population)
        best_invi = population[0]

        if best_invi[2] > 20*(n**0.5):
            return population

        if _ % 100 == 0:
            print(f'Loop: {_},\tFitness: {best_invi[0]},\tDistance: {totalDistance(best_invi[1])},\tBlinding constrains: {nOfBlindingConstrains(best_invi[1])}')

    return population

total_distance = 0
total_time = 0
total_bCons = 0
for n_seed in range(10):
    random.seed(n_seed)
    cities = np.arange(1, 2*n+1)
    population = initPopulation(cities, size=100) # Khởi tạo quần thể đầu tiên
    t1 = time.time()
    pop = GA(population=population, crossover_func=multiPointCrossover, lenCities=len(cities))
    total_distance += totalDistance(pop[0][1])
    total_time += (time.time() - t1)
    total_bCons += nOfBlindingConstrains(pop[0][1])

print(f"Objective value: {total_distance/10}")
# print(f"Solution: {[0] + pop[0][1]}")
print(f"Number of blinding constrains: {total_bCons/10}")
print(f"Time: {total_time/10}")

Loop: 0,	Fitness: 284.0,	Distance: 284,	Blinding constrains: 0
Loop: 100,	Fitness: 146.0,	Distance: 146,	Blinding constrains: 0
Loop: 0,	Fitness: 260.0,	Distance: 260,	Blinding constrains: 0
Loop: 0,	Fitness: 239.0,	Distance: 239,	Blinding constrains: 0
Loop: 100,	Fitness: 121.0,	Distance: 121,	Blinding constrains: 0
Loop: 0,	Fitness: 286.0,	Distance: 286,	Blinding constrains: 0
Loop: 100,	Fitness: 155.0,	Distance: 155,	Blinding constrains: 0
Loop: 0,	Fitness: 301.05019147290193,	Distance: 166,	Blinding constrains: 1
Loop: 100,	Fitness: 133.0,	Distance: 133,	Blinding constrains: 0
Loop: 0,	Fitness: 197.0,	Distance: 197,	Blinding constrains: 0
Loop: 100,	Fitness: 138.0,	Distance: 138,	Blinding constrains: 0
Loop: 0,	Fitness: 236.0,	Distance: 236,	Blinding constrains: 0
Loop: 100,	Fitness: 146.0,	Distance: 146,	Blinding constrains: 0
Loop: 0,	Fitness: 220.0,	Distance: 220,	Blinding constrains: 0
Loop: 100,	Fitness: 126.0,	Distance: 126,	Blinding constrains: 0
Loop: 200,	Fitness: 108.0,	D

In [ ]:
# def multiPointCrossover(father1, father2, n_points):
#     points = sorted(random.sample(father1, n_points))
#     length = len(father1)
#     if n_points%2 != 0:
#         points.append(length)
#         n_points += 1

#     new_inv1 = father2
#     new_inv2 = father1
#     cl1, cl2 = [], []
#     for idx in range(0, n_points, 2):
#         cl1 += father2[points[idx]:points[idx+1]]
#         cl2 += father1[points[idx]:points[idx+1]]

#     for idx in range(0, n_points, 2):
#         i1, i2 = points[idx], points[idx+1]
#         i = 0
#         for inv in father1:
#             if i == i2 - i1:
#                 break
#             if inv in cl1:
#                 new_inv1[i1+i] = inv
#                 cl1.remove(inv)
#                 i += 1
#     for idx in range(0, n_points, 2):
#         i1, i2 = points[idx], points[idx+1]
#         i = 0
#         for inv in father2:
#             if i == i2 - i1:
#                 break
#             if inv in cl2:
#                 new_inv2[i1+i] = inv
#                 cl2.remove(inv)
#                 i += 1

#     return new_inv1, new_inv2